# **NLP3_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """ A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased. """

In [5]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [6]:
print(response1.choices[0].message.content)

Parameters:
- T = Total number of units needed = 239,600,480
- C = The number of units each vendor can supply: C = {C_A = 33,000,000; C_B = 160,000,000; C_C = 165,600,000; C_D = 12,000,000; C_E = 77,000,000}
- A_pc = A's per-unit price and setUp cost
- B_pc[X] = B's per-unit price and setUp cost when purchasing X units
- C_pc = C's per-unit price and setUp cost
- D_pc = D's per-unit price and setUp cost
- E_pc[X] = E's per-unit price and setUp cost when purchasing X units

Decision Variables:
- A, B, C, D, E denote the quantity from each vendor.

Objective Function:
Minimize: (A * A_pc + B_pc[B] * B + C * C_pc + D * D_pc + E_pc[E] * E)

Constraints:
- Supply Constraint:  A + B + C + D + E  =  T
- Vendor's Capacity Constraints:  
  - A ≤ C_A
  - B ≤ C_B
  - C ≤ C_C
  - D ≤ C_D
  - E ≤ C_E
- Vendor B supply structure: If B>0 then B >= 22,000,000
- Vendor E supply structure: If E>0 and E<=42,000,000 then no setup cost, else setup cost added.

This model excludes other specifics such as in

### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [8]:
Markdown(response2.choices[0].message.content)

Sample Python code with Pyomo for the given optimization problem could be:

```python
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Set sample data
A_pc = 20
B_pc = 30
C_pc = 40
D_pc = 50
E_pc = 60
setup_cost = 1000  # this is the setup cost for vendor E
vendors = ['A', 'B', 'C', 'D', 'E']
cap = {'A': 33000000, 'B': 160000000, 'C': 165600000, 'D': 12000000, 'E': 77000000}

# Initialize the variables [A, B, C, D, E]
model.A = Var(within=NonNegativeReals)
model.B = Var(within=NonNegativeReals)
model.C = Var(within=NonNegativeReals)
model.D = Var(within=NonNegativeReals)
model.E = Var(within=NonNegativeReals)

# Create the objective function
model.cost = Objective(expr=(model.A * A_pc + model.B * B_pc + model.C * C_pc + model.D * D_pc + 
                             (model.E * E_pc if model.E <= 42000000 else model.E * E_pc + setup_cost)),
                       sense=minimize)

# Add the supply constraint
model.supply = Constraint(expr=model.A + model.B + model.C + model.D + model.E == 239600480)

# Add the vendor's capacity constraints
model.vendor_capacity = ConstraintList()
for vendor in vendors:
  model.vendor_capacity.add(getattr(model, vendor) <= cap[vendor])

# Vendor B and E supply structure constraint
model.vendor_min_B = Constraint(expr=model.B >= 22000000 if model.B > 0 else model.B >= 0)
model.vendor_E_setup = Constraint(expr=model.E >= 42000000 if model.E > 0 else model.E >= 0)

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# Print the values of the decision variables
print("Values of Decision Variables : ")
print("A = ", model.A())
print("B = ", model.B())
print("C = ", model.C())
print("D = ", model.D())
print("E = ", model.E())
```
Please note that this model code has some assumptions (such as setup_cost) due to lack of specific details in the original problem.

### **Run the code Generated by GPT4**

In [9]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 8.1 MB/s eta 0:00:00


In [10]:

_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [12]:
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Set sample data
A_pc = 20
B_pc = 30
C_pc = 40
D_pc = 50
E_pc = 60
setup_cost = 1000  # this is the setup cost for vendor E
vendors = ['A', 'B', 'C', 'D', 'E']
cap = {'A': 33000000, 'B': 160000000, 'C': 165600000, 'D': 12000000, 'E': 77000000}

# Initialize the variables [A, B, C, D, E]
model.A = Var(within=NonNegativeReals)
model.B = Var(within=NonNegativeReals)
model.C = Var(within=NonNegativeReals)
model.D = Var(within=NonNegativeReals)
model.E = Var(within=NonNegativeReals)

# Create the objective function
model.cost = Objective(expr=(model.A * A_pc + model.B * B_pc + model.C * C_pc + model.D * D_pc +
                             (model.E * E_pc if model.E <= 42000000 else model.E * E_pc + setup_cost)),
                       sense=minimize)

# Add the supply constraint
model.supply = Constraint(expr=model.A + model.B + model.C + model.D + model.E == 239600480)

# Add the vendor's capacity constraints
model.vendor_capacity = ConstraintList()
for vendor in vendors:
  model.vendor_capacity.add(getattr(model, vendor) <= cap[vendor])

# Vendor B and E supply structure constraint
model.vendor_min_B = Constraint(expr=model.B >= 22000000 if model.B > 0 else model.B >= 0)
model.vendor_E_setup = Constraint(expr=model.E >= 42000000 if model.E > 0 else model.E >= 0)

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# Print the values of the decision variables
print("Values of Decision Variables : ")
print("A = ", model.A())
print("B = ", model.B())
print("C = ", model.C())
print("D = ", model.D())

PyomoException: Cannot convert non-constant Pyomo expression (E  <=  42000000) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**